In [1]:
from langchain.indexes import GraphIndexCreator
# from langchain_openai import OpenAI
from langchain_community.llms import Ollama

import networkx as nx


from dotenv import load_dotenv

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

from operator import itemgetter

from langchain_community.graphs.networkx_graph import NetworkxEntityGraph
from langchain_community.graphs.networkx_graph import KnowledgeTriple

from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [5]:
load_dotenv()

True

In [6]:
model = Ollama(model="llama2")

In [7]:
output_parser = StrOutputParser()

In [8]:
# read graph as entity graph
G = NetworkxEntityGraph()

In [9]:
G.add_triple(KnowledgeTriple('Charging Station', 'connects to', 'N1'))
G.add_triple(KnowledgeTriple('N1', 'connects to', 'Mensa'))
G.add_triple(KnowledgeTriple('N1', 'connects to', 'N2'))
G.add_triple(KnowledgeTriple('Mensa', 'connects to', 'N3'))
G.add_triple(KnowledgeTriple('N2', 'connects to', 'N3'))
G.add_triple(KnowledgeTriple('N2', 'connects to', 'Blood Storage'))
G.add_triple(KnowledgeTriple('N2', 'connects to', 'N4'))
G.add_triple(KnowledgeTriple('N3', 'connects to', 'N4'))
G.add_triple(KnowledgeTriple('N3', 'connects to', 'Waste Disposal'))

In [21]:
[(triple[0], triple[1]) for triple in G.get_triples()]


[('Charging Station', 'N1'),
 ('N1', 'Mensa'),
 ('N1', 'N2'),
 ('Mensa', 'N3'),
 ('N2', 'N3'),
 ('N2', 'Blood Storage'),
 ('N2', 'N4'),
 ('N3', 'N4'),
 ('N3', 'Waste Disposal')]

In [26]:
template = f"Answer the question based only on the following context: We have a directed graph. It contains these edges: {[(triple[0], triple[1]) for triple in G.get_triples()]} \n Question: {{question}}"

In [27]:
prompt = ChatPromptTemplate.from_template(template)

In [28]:
# create chain
chain = ({"question": RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser()
)

In [29]:
chain.invoke("What are the options to get from Charging Station to Waste Disposal? Can you list the routs?")

'\nGiven the provided graph, there are two possible routes to get from Charging Station to Waste Disposal:\n\n1. N1 -> Mensa -> N2 -> Blood Storage -> Waste Disposal\n2. N1 -> N2 -> Blood Storage -> Waste Disposal\n\nSo, the options to get from Charging Station to Waste Disposal are:\n\n* Go through Mensa and then take the path from Blood Storage to Waste Disposal.\n* Go directly from Charging Station to Blood Storage and then take the path from Blood Storage to Waste Disposal.'

In [30]:
chain.invoke("Assume that node N2 is not available any more. Could you update the list of edges?")

"\nOf course! If node N2 is no longer available, we can update the list of edges as follows:\n\n* The edge ('Charging Station', 'N1') remains unchanged.\n* The edge ('N1', 'Mensa') remains unchanged.\n* The edge ('N1', 'N2') is removed.\n* The edge ('Mensa', 'N3') remains unchanged.\n* The edge ('N2', 'Blood Storage') becomes an edge between node N1 and node Blood Storage.\n* The edge ('N2', 'Waste Disposal') becomes an edge between node N1 and node Waste Disposal.\n\nSo, the updated list of edges would be:\n\n* ('Charging Station', 'N1')\n* ('N1', 'Mensa')\n* ('Mensa', 'N3')\n* ('Blood Storage', 'N1')\n* ('Waste Disposal', 'N1')"